In [1]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import pytz

mt5 = metatrader.connect()

symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H1

trade_volume = 100000
commission = 0.00005



Connected to Metatrader successfully.


In [2]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0,10000))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume
0,2022-03-21 08:00:00,1.10412,1.10439,1.10376,1.10416,1334,1,0,100000
1,2022-03-21 09:00:00,1.10416,1.10467,1.10344,1.10382,1249,0,0,100000
2,2022-03-21 10:00:00,1.10381,1.10557,1.10348,1.10507,4178,0,0,100000
3,2022-03-21 11:00:00,1.10508,1.10625,1.10463,1.10565,5799,0,0,100000
4,2022-03-21 12:00:00,1.10566,1.10701,1.10513,1.10570,5156,0,0,100000
...,...,...,...,...,...,...,...,...,...
9995,2023-10-24 19:00:00,1.05938,1.06022,1.05834,1.05867,2682,0,0,100000
9996,2023-10-24 20:00:00,1.05868,1.05941,1.05865,1.05888,3073,0,0,100000
9997,2023-10-24 21:00:00,1.05887,1.05909,1.05845,1.05884,2165,0,0,100000
9998,2023-10-24 22:00:00,1.05884,1.05936,1.05870,1.05918,1312,0,0,100000


In [3]:
bars['sma_100'] = bars['close'].ewm(span=20).mean()
bars['sma_10'] = bars['close'].ewm(span=10).mean()
bars['hour'] = bars['time'].dt.hour
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour
0,2022-03-21 08:00:00,1.10412,1.10439,1.10376,1.10416,1334,1,0,100000,1.104160,1.104160,8
1,2022-03-21 09:00:00,1.10416,1.10467,1.10344,1.10382,1249,0,0,100000,1.103982,1.103973,9
2,2022-03-21 10:00:00,1.10381,1.10557,1.10348,1.10507,4178,0,0,100000,1.104381,1.104414,10
3,2022-03-21 11:00:00,1.10508,1.10625,1.10463,1.10565,5799,0,0,100000,1.104747,1.104821,11
4,2022-03-21 12:00:00,1.10566,1.10701,1.10513,1.10570,5156,0,0,100000,1.104978,1.105073,12
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-10-24 19:00:00,1.05938,1.06022,1.05834,1.05867,2682,0,0,100000,1.063139,1.061817,19
9996,2023-10-24 20:00:00,1.05868,1.05941,1.05865,1.05888,3073,0,0,100000,1.062734,1.061283,20
9997,2023-10-24 21:00:00,1.05887,1.05909,1.05845,1.05884,2165,0,0,100000,1.062363,1.060839,21
9998,2023-10-24 22:00:00,1.05884,1.05936,1.05870,1.05918,1312,0,0,100000,1.062060,1.060537,22


In [4]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 1 <= x['hour'] <= 24:

        if x['sma_10'] > x['sma_100']:
            return 1
        elif x['sma_10'] < x['sma_100']:
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour,signal,prev_price,price_change,signal_change,commission
0,2022-03-21 08:00:00,1.10412,1.10439,1.10376,1.10416,1334,1,0,100000,1.104160,1.104160,8,0,NaN,NaN,NaN,NaN
1,2022-03-21 09:00:00,1.10416,1.10467,1.10344,1.10382,1249,0,0,100000,1.103982,1.103973,9,-1,1.10416,-0.00034,1.0,5.0
2,2022-03-21 10:00:00,1.10381,1.10557,1.10348,1.10507,4178,0,0,100000,1.104381,1.104414,10,1,1.10382,0.00125,2.0,10.0
3,2022-03-21 11:00:00,1.10508,1.10625,1.10463,1.10565,5799,0,0,100000,1.104747,1.104821,11,1,1.10507,0.00058,0.0,0.0
4,2022-03-21 12:00:00,1.10566,1.10701,1.10513,1.10570,5156,0,0,100000,1.104978,1.105073,12,1,1.10565,0.00005,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-10-24 19:00:00,1.05938,1.06022,1.05834,1.05867,2682,0,0,100000,1.063139,1.061817,19,-1,1.05937,-0.00070,0.0,0.0
9996,2023-10-24 20:00:00,1.05868,1.05941,1.05865,1.05888,3073,0,0,100000,1.062734,1.061283,20,-1,1.05867,0.00021,0.0,0.0
9997,2023-10-24 21:00:00,1.05887,1.05909,1.05845,1.05884,2165,0,0,100000,1.062363,1.060839,21,-1,1.05888,-0.00004,0.0,0.0
9998,2023-10-24 22:00:00,1.05884,1.05936,1.05870,1.05918,1312,0,0,100000,1.062060,1.060537,22,-1,1.05884,0.00034,0.0,0.0


In [5]:
bars3 = bars2.dropna().copy()
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()
bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
1,2022-03-21 09:00:00,1.10416,1.10467,1.10344,1.10382,1249,0,0,100000,1.103982,1.103973,9,-1,1.10416,-0.00034,1.0,5.0,34.0,34.0,29.0
2,2022-03-21 10:00:00,1.10381,1.10557,1.10348,1.10507,4178,0,0,100000,1.104381,1.104414,10,1,1.10382,0.00125,2.0,10.0,125.0,159.0,144.0
3,2022-03-21 11:00:00,1.10508,1.10625,1.10463,1.10565,5799,0,0,100000,1.104747,1.104821,11,1,1.10507,0.00058,0.0,0.0,58.0,217.0,202.0
4,2022-03-21 12:00:00,1.10566,1.10701,1.10513,1.10570,5156,0,0,100000,1.104978,1.105073,12,1,1.10565,0.00005,0.0,0.0,5.0,222.0,207.0
5,2022-03-21 13:00:00,1.10570,1.10584,1.10448,1.10453,3484,0,0,100000,1.104883,1.104932,13,1,1.10570,-0.00117,0.0,0.0,-117.0,105.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-10-24 19:00:00,1.05938,1.06022,1.05834,1.05867,2682,0,0,100000,1.063139,1.061817,19,-1,1.05937,-0.00070,0.0,0.0,70.0,94447.0,86182.0
9996,2023-10-24 20:00:00,1.05868,1.05941,1.05865,1.05888,3073,0,0,100000,1.062734,1.061283,20,-1,1.05867,0.00021,0.0,0.0,-21.0,94426.0,86161.0
9997,2023-10-24 21:00:00,1.05887,1.05909,1.05845,1.05884,2165,0,0,100000,1.062363,1.060839,21,-1,1.05888,-0.00004,0.0,0.0,4.0,94430.0,86165.0
9998,2023-10-24 22:00:00,1.05884,1.05936,1.05870,1.05918,1312,0,0,100000,1.062060,1.060537,22,-1,1.05884,0.00034,0.0,0.0,-34.0,94396.0,86131.0


In [6]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="SP500 NY session Backtest MA Trendfollowing")


c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
